In [1]:
# 環境変数の読み込み
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

# アシスタントを生成
assistant = client.beta.assistants.create(
    instructions="あなたはITエンジニアの優秀なアシスタントです。",
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "find_file_by_word",
                "description": "特定のワードを含むファイルを検索する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_word": {
                            "type": "string",
                            "description": "検索ワード"
                        }
                    },
                    "required": ["search_word"]
                }
            }
        },
    ],
)
assistant

Assistant(id='asst_mqJHgtV5axQmIVQ7noOg9Hsi', created_at=1720870548, description=None, instructions='あなたはITエンジニアの優秀なアシスタントです。', metadata={}, model='gpt-4o', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='find_file_by_word', description='特定のワードを含むファイルを検索する', parameters={'type': 'object', 'properties': {'search_word': {'type': 'string', 'description': '検索ワード'}}, 'required': ['search_word']}), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [3]:
assistant = client.beta.assistants.retrieve('asst_mqJHgtV5axQmIVQ7noOg9Hsi')
assistant

Assistant(id='asst_mqJHgtV5axQmIVQ7noOg9Hsi', created_at=1720870548, description=None, instructions='あなたはITエンジニアの優秀なアシスタントです。', metadata={}, model='gpt-4o', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='find_file_by_word', description='特定のワードを含むファイルを検索する', parameters={'type': 'object', 'properties': {'search_word': {'type': 'string', 'description': '検索ワード'}}, 'required': ['search_word']}), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [25]:
# スレッドを生成
thread = client.beta.threads.create()
thread

Thread(id='thread_fqLTlDjIAw1CjdsyL6xtVRJz', created_at=1720871778, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [26]:
# スレッドにメッセージを追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="「sample」が含まれるファイルを検索して"
)
message

Message(id='msg_VdlG9DMBY9bjd4oTFbnBQcU6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='「sample」が含まれるファイルを検索して'), type='text')], created_at=1720871780, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fqLTlDjIAw1CjdsyL6xtVRJz')

In [27]:
# スレッドを実行
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="Please address the user as Jane Doe. The user has a premium account."
)
run

Run(id='run_Js2Pb5Ay167zBTKk3erMXJ0C', assistant_id='asst_mqJHgtV5axQmIVQ7noOg9Hsi', cancelled_at=None, completed_at=None, created_at=1720871785, expires_at=1720872385, failed_at=None, incomplete_details=None, instructions='あなたはITエンジニアの優秀なアシスタントです。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_fqLTlDjIAw1CjdsyL6xtVRJz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='find_file_by_word', description='特定のワードを含むファイルを検索する', parameters={'type': 'object', 'properties': {'search_word': {'type': 'string', 'description': '検索ワード'}}, 'required': ['search_word']}), type='function')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [28]:
# スレッドの実行結果を取得
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)
print(run.required_action)

requires_action
RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_dws2s7D3z79tOS9tAFOsYTdK', function=Function(arguments='{"search_word":"sample"}', name='find_file_by_word'), type='function')]), type='submit_tool_outputs')


In [29]:
if run.status == 'requires_action':
    tool_outputs = []
    calling_tool_list = run.required_action.submit_tool_outputs.tool_calls
    for calling_tool in calling_tool_list:
        calling_function = calling_tool.function
        print(calling_function)
        if calling_function.name == 'find_file_by_word':
            # 現在のディレクトリから「search_word」をファイルの内容に含むファイルを検索
            import os, json
            arguments = json.loads(calling_function.arguments)
            print(arguments)
            search_word = arguments['search_word']
            files = [f for f in os.popen(f'grep -lr {search_word} ./docs').read().split('\n') if f]
            print(files)
            tool_outputs.append({
              "tool_call_id": calling_tool.id,
              "output": ', '.join(files)
            })


    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs=tool_outputs
    )

Function(arguments='{"search_word":"sample"}', name='find_file_by_word')
{'search_word': 'sample'}
['./docs/sample.txt']


In [30]:
print(run.status)
print(run.required_action)

completed
None


In [31]:
response = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(response.data)

[Message(id='msg_oz2WTuOUH7GB4AyKXr9BriPO', assistant_id='asst_mqJHgtV5axQmIVQ7noOg9Hsi', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='「sample」が含まれるファイルは以下の通りです：\n- `./docs/sample.txt`'), type='text')], created_at=1720871794, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Js2Pb5Ay167zBTKk3erMXJ0C', status=None, thread_id='thread_fqLTlDjIAw1CjdsyL6xtVRJz'), Message(id='msg_VdlG9DMBY9bjd4oTFbnBQcU6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='「sample」が含まれるファイルを検索して'), type='text')], created_at=1720871780, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fqLTlDjIAw1CjdsyL6xtVRJz')]


In [32]:
print(response.data[0].content[0].text.value)

「sample」が含まれるファイルは以下の通りです：
- `./docs/sample.txt`
